# NIH MEG Workshop 2023 Intro

## This should not be used without modification -- copy from previous lecture

## ___________________________________________________

## General Import and settings

In [ ]:
import mne
import os
import numpy as np
import warnings
import copy

warnings.filterwarnings('ignore', '.*Use of `point_arrays` is deprecated.*',)
warnings.filterwarnings('ignore', '.*Matplotlib.*',) 

### Define Dataset and paths

In [ ]:
topdir=os.getcwd()
subjects_dir = f'{topdir}/SUBJECTS_DIR'
output_dir = f'{topdir}/outputs'
if not os.path.exists(output_dir): os.mkdir(output_dir)

meg_fname = f'{topdir}/sub-ON39099/ses-01/meg/sub-ON39099_ses-01_task-airpuff_run-01_meg.ds'

In [ ]:
subjid = os.path.basename(meg_fname).split('_')[0]
print(f'Subject ID is: {subjid}')

### Install code to convert NIH formatted fiducials (Brainsight, Afni) to MNE transformation matrix

In [ ]:
#  This is a call to the BASH shell 
! if [[ -z $(pip list | grep nih2mne) ]] ; then pip install ./additional_code/nih_to_mne; fi
! if [[ -z $(pip list | grep hv-proc) ]] ; then pip install ./additional_code/hv_proc; pip install ./additional_code/pyctf-lite; fi

### Verify data is present - if this fails something is wrong

In [ ]:
assert os.path.exists(output_dir)
assert os.path.exists(subjects_dir)
assert os.path.exists(meg_fname)

# Start the preprocessing

In [ ]:
raw = mne.io.read_raw_ctf(meg_fname, clean_names=True, system_clock='ignore')

In [ ]:
raw.info

#### 3rd Order Gradiometers applied - In our case 3rd order compensation is set by default during acquisition

In [ ]:
raw.compensation_grade

In [ ]:
%matplotlib inline
_=raw.copy().apply_gradient_compensation(1).plot(n_channels=10)
print('3rd Order Gradient applied')
_=raw.plot(n_channels=10)

In [ ]:
raw.load_data()
raw._data.shape

In [ ]:
raw

In [ ]:
raw.ch_names

### Selecting channels

In [ ]:
# Also look at raw.pick_channels and raw.pick
raw.pick_types?

In [ ]:
raw.pick_types(meg=True, eeg=False, misc=True)._data.shape

In [ ]:
raw.ch_names.index('UADC001')

In [ ]:
#General plotting using python plotting
%matplotlib inline
import pylab
pylab.plot(raw._data[301, 0:10000])

### Select Misc channels to view triggers and plot using MNE Python

In [ ]:
# Select the stimulus and response channels (if present)
# If the .copy() is not used, the raw data itself will drop all channels except the selected channels
misc1 = raw.copy().pick_types(meg=False, eeg=False, misc=True)

### Somatosensory task - Airpuff Stimulus (Stim) vs. Missing Stim 
### UPPT001 codes stimuli; UADC001 communicates to mechanical airpuff delivery system

In [ ]:
%matplotlib qt
#Set scalings by data type - eg scalings={'mag':300} for evoked meg data
misc1.plot(scalings={'misc':10}, duration=4)  

In [ ]:
front = [i for i in raw.ch_names if i[2]=='F']
print(f'There are {len(front)} frontal channels')

In [ ]:
front_data = raw.copy().pick_channels(front)

In [ ]:
%matplotlib qt
front_data.plot()

### Sensor level preprocessing

In [ ]:
raw = raw.load_data() 

In [ ]:
raw.resample(300)

raw.filter(1, None) 

In [ ]:
%matplotlib inline
_ = raw.plot_psd()

In [ ]:
raw.notch_filter([60,120])
_ = raw.plot_psd()

### Smaller window of spectrum

In [ ]:
%matplotlib inline
_ = raw.plot_psd(fmin=3, fmax=45, n_overlap=.5)

# Creating Events and Epochs from the data

In [ ]:
raw.annotations  #These have been read from the MarkerFile.mrk upon reading 

In [ ]:
events, event_ids = mne.events_from_annotations(raw) 

In [ ]:
# Dictionary that maps event name to event code
event_ids

In [ ]:
events    # [Sample , duration, stim_code]

In [ ]:
%matplotlib inline
_ = mne.viz.plot_events(events, sfreq=raw.info['sfreq'], event_id=event_ids)

## Epochs: Time windows of data around each event 

In [ ]:
#At time 0ms the stimuli is delivered
tmin=-0.1  #100ms before event
tmax=0.3   #300ms after event

#Calculate all epochs with no rejection
epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax, reject_tmax=0,
                preload=True, baseline=(tmin, 0))


In [ ]:
epochs._data.shape

In [ ]:
%matplotlib qt
_=epochs.plot(n_channels=10)

### Rejecting bad epochs using amplitude thresholding

In [ ]:
reject_dict = {'mag':2500e-15}
epochs.drop_bad(reject=reject_dict)

### Epochs data size: Epochs X Channels X TimeSteps

In [ ]:
epochs._data.shape

### Evoked data averages over epochs: Channels X TimeSteps

In [ ]:
evoked = epochs['2'].average()
evoked._data.shape  #Also removes 4 response/stim/clock channels

### Butterfly plot of evoked brain activity

In [ ]:
%matplotlib inline
_=evoked.plot()

### Plot evoked dataset over time

In [ ]:
_=evoked.plot_topomap(times=np.arange(0,.1, 0.01))

### Effect of number of samples on data average

In [ ]:
from matplotlib import pyplot as plt

stim_epochs=epochs['2']

%matplotlib qt
fig = plt.Figure()
for idx,num in enumerate([1,2,4,8,16]):
    ax = plt.subplot(3,2,idx+1)
    evoked = stim_epochs[0:-1:num].average(method='mean')
    evoked.plot(axes=ax, ylim={'mag':[-200,200]})
plt.tight_layout()
plt.show()


In [ ]:
from matplotlib import pyplot as plt

stim_epochs=epochs['2']

%matplotlib qt
fig = plt.Figure()
for idx,num in enumerate([1,2,4,8,16]):
    ax = plt.subplot(3,2,idx+1)
    evoked = stim_epochs[0:-1:num].average(method='mean')
    evoked.plot_topomap(times=[0.040], colorbar=False, vmin=-100, vmax=100, axes=ax)
    plt.title(f'40ms - Number Ave:{evoked.nave}')
plt.tight_layout()
plt.show()layout()
plt.show()layout()
plt.show()
